In [2]:
import numpy as np, pandas as pd, polars as pl
from enum import Enum
from sklearn import preprocessing as skp
from sklearn.model_selection import KFold

import sklearn.linear_model as skl
from sklearn.ensemble import HistGradientBoostingRegressor as gbr
import xgboost as xgb
from sklearn.metrics import root_mean_squared_error as rmse

In [3]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 750000 non-null  object 
 1   Episode_Title                750000 non-null  object 
 2   Episode_Length_minutes       662907 non-null  float64
 3   Genre                        750000 non-null  object 
 4   Host_Popularity_percentage   750000 non-null  float64
 5   Publication_Day              750000 non-null  object 
 6   Publication_Time             750000 non-null  object 
 7   Guest_Popularity_percentage  603970 non-null  float64
 8   Number_of_Ads                749999 non-null  float64
 9   Episode_Sentiment            750000 non-null  object 
 10  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), object(6)
memory usage: 68.7+ MB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250000 entries, 750000 to 999999
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 250000 non-null  object 
 1   Episode_Title                250000 non-null  object 
 2   Episode_Length_minutes       221264 non-null  float64
 3   Genre                        250000 non-null  object 
 4   Host_Popularity_percentage   250000 non-null  float64
 5   Publication_Day              250000 non-null  object 
 6   Publication_Time             250000 non-null  object 
 7   Guest_Popularity_percentage  201168 non-null  float64
 8   Number_of_Ads                250000 non-null  float64
 9   Episode_Sentiment            250000 non-null  object 
dtypes: float64(4), object(6)
memory usage: 21.0+ MB


In [6]:
categorical_map = {
        'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6, # Day

        'Morning': 0, 'Afternoon': 1, 'Evening': 2, 'Night': 3,                                            # Time

        'Negative': 0, 'Neutral': 1, 'Positive': 2,                                                        # Sentiment

        'True Crime': 0, 'Comedy': 1, 'Education': 2, 'Technology': 3, 'Health': 4, 'News': 5, 'Music': 6, # Genre
        'Sports': 7, 'Business': 8, 'Lifestyle': 9
        }

categories = ['Genre', 'Episode_Title', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']

In [7]:
train.drop('Podcast_Name', inplace=True, axis=1)
test.drop('Podcast_Name', inplace=True, axis=1)

In [8]:
train['Episode_Title'] = train['Episode_Title'].str[8:]

In [9]:
train = train.replace(categorical_map)

In [10]:
for c in categories:
    train[c] = train[c].astype('category')

In [11]:
x = train.drop('Listening_Time_minutes', axis=1)
y = train['Listening_Time_minutes']

In [27]:
kf = KFold(7, shuffle=True, random_state=55)

In [29]:
for train_idx, val_idx in kf.split(x,y):
    x_t, y_t = x.iloc[train_idx], y.iloc[train_idx]
    x_v, y_v = x.iloc[val_idx], y.iloc[val_idx]
    model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric = 'rmse', enable_categorical = True, device = 'cuda', tree_method="hist")
    model.fit(x_t, y_t, eval_set=[(x_v,y_v)]) 
    
    y_p = model.predict(x_v)

[0]	validation_0-rmse:21.25810
[1]	validation_0-rmse:17.65282
[2]	validation_0-rmse:15.56669
[3]	validation_0-rmse:14.42861
[4]	validation_0-rmse:13.82413
[5]	validation_0-rmse:13.50647
[6]	validation_0-rmse:13.34533
[7]	validation_0-rmse:13.25843
[8]	validation_0-rmse:13.21172
[9]	validation_0-rmse:13.18459
[10]	validation_0-rmse:13.16566
[11]	validation_0-rmse:13.15285
[12]	validation_0-rmse:13.14521
[13]	validation_0-rmse:13.13537
[14]	validation_0-rmse:13.13095
[15]	validation_0-rmse:13.12776
[16]	validation_0-rmse:13.12137
[17]	validation_0-rmse:13.11637
[18]	validation_0-rmse:13.11503
[19]	validation_0-rmse:13.10981
[20]	validation_0-rmse:13.10841
[21]	validation_0-rmse:13.10683
[22]	validation_0-rmse:13.10571
[23]	validation_0-rmse:13.10463
[24]	validation_0-rmse:13.10376
[25]	validation_0-rmse:13.10296
[26]	validation_0-rmse:13.10114
[27]	validation_0-rmse:13.09919
[28]	validation_0-rmse:13.09694
[29]	validation_0-rmse:13.09553
[30]	validation_0-rmse:13.09447
[31]	validation_0-